# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 2 from hw3-t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [7]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [8]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [9]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [10]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    def add_source_and_sink(orig_graph):
        """Returns a new graph with s,t nodes added and edges from s to supply nodes and from demand nodes to t"""
        graph = orig_graph.copy() # always work with a copy, modifying original is a bad practice
        s = 's_long_string_not_to_conflict_with_any_preexisting_node'
        t = 't_long_string_not_to_conflict_with_any_preexisting_node'
        for node, props in graph.nodes(data=True):
            if props['demand'] < 0:
                graph.add_edge(s, node, capacity=abs(props['demand']))
            elif props['demand'] > 0:
                graph.add_edge(node, t, capacity=props['demand'])
        return graph, s, t
    
    def remove_source_and_sink_from_flow(flow, s, t):
        """Removes nodes s and t from flow (from top dict and inside dicts)"""
        def without(d, keys):
            return { u:d[u] for u in d if not u in keys }
        return {k:without(flow[k], [s,t]) for k in without(flow, [s,t])}
    
    def full_capacity(g, s):
        """Full capacity of edges coming out of s"""
        return sum([props['capacity'] for _, _, props in g.edges(s, data=True)])
    
    def to_dict_of_dicts(g, data=0):
        """Returns dict of dicts in the format we want our flow structure to be with 0 values for flows"""
        result = {}
        for u,neighbours in g.adjacency_iter():
                result[u]=dict.fromkeys(neighbours, data)
        return result
    
    def flow_value(flow, s):
        """Returns flow value, which is the sum of flows out of s"""
        return sum(flow[s].values())

    def residual_graph(g, flow):
        """
        Returns residual graph for graph g and flow.
        Each edge will have 'capacity' (number) and 'forward' (Bool) properties.
        """
        residual = nx.DiGraph()
        for u, v, props in g.edges(data=True):
            # if the flow less then capacity, add forward edge with capacity == the diff (residual)
            if flow[u][v] < props['capacity']:
                residual.add_edge(u, v, capacity=(props['capacity']-flow[u][v]), forward=True)
            # if the flow is more the 0, add backward edge with capacity = the flow
            if flow[u][v] > 0:
                residual.add_edge(v, u, capacity=flow[u][v], forward=False)
        return residual
    
    def BFS(g, s):
        """
        Implementation of BFS algo.
        Returns a distances of nodes from s and parent for each node (with None value if node is un-reachable)
        """
        discovered = {n:False for n in g.nodes()}
        distance = {n:float("inf") for n in g.nodes()}
        parent = {n:None for n in g.nodes()}
        q = [] # using list as a queue
        discovered[s] = True
        distance[s] = 0
        parent[s] = None
        q.append(s)
        while len(q) > 0:
            u = q.pop(0)
            for v in g.adj[u]:
                if not discovered[v]:
                    discovered[v] = True
                    distance[v] = distance[u] + 1
                    parent[v] = u
                    q.append(v)
        return distance, parent

    def s_t_path(g, s, t):
        """
        Returns a path from s to t. If multiple paths exist the choice non-deterministic
        """
        _, path_parents = BFS(g, s)
        if path_parents[t] is None:
            return None
        node = t
        path = [node]
        while node != s:
            node = path_parents[node]
            path.insert(0, node)
        return path
    
    def path_edges(path):
        return zip(path[:-1], path[1:])
    
    def path_capacity(path, g):
        """Returns min value of capacity along path edges in graph g"""
        return min([g[u][v]['capacity'] for u,v in path_edges(path)])
    
    def augment_flow(flow, g, path):
        # would be best to deepcopy flow here instead of updating inplace
        c_P = path_capacity(path, g)
        for u,v in path_edges(path):
            if g[u][v]['forward']:
                flow[u][v] = flow[u][v] + c_P
            else:
                flow[v][u] = flow[v][u] - c_P
        return flow
    
    def maximum_flow(G, s, t):
        """
        Implements Ford-Fulkerson algorithm.
        Args: G - graph, s, t - nodes
        Returns: flow_value, flow_dict - dict of dict where flow_dict[from][to] = flow value on the edge (from,to)
        """
        # Initialize flow structure with 0 values
        flow = to_dict_of_dicts(G, data=0)
        R = residual_graph(G, flow)
        path = s_t_path(R, s, t)
        while path:
            flow = augment_flow(flow, R, path)
            R = residual_graph(G, flow)
            path = s_t_path(R, s, t)
        return flow_value(flow, s), flow
        
    augmented_G, s, t = add_source_and_sink(graph)
    # Commented out standard NetworkX maximum_flow implementation
    #flow_value, flow_dict = nx.maximum_flow(augmented_G, s, t)
    # Using hand-coded maximum_flow implementation
    max_flow_value, flow_dict = maximum_flow(augmented_G, s, t)
    if max_flow_value < full_capacity(augmented_G, s):
        raise nx.NetworkXUnfeasible("There is no flow satisfying the demands")
    return remove_source_and_sink_from_flow(flow_dict, s, t)

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [11]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # Assumption: flow keys should be a set of all nodes.
    div = {node:0 for node in flow.keys()}
    for u, neighbors in flow.iteritems():
        for v, f in neighbors.iteritems():
            div[u] = div[u] - f
            div[v] = div[v] + f
    return div
    

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [12]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
